# Assignment 1

## Predicting: How Weather and Seasonal Event Shaped NYC Taxi Demand: Oct 2024 - Apr 2025

+ Inspecting weather and holidays on the demand and tips of the iconic yellow taxi

## Download Data

### Download weather stat Oct 2024 - Apr 2025


Weather_URL = https://www.ncei.noaa.gov/access/search/data-search/global-hourly?startDate=2024-10-01T00:00:00&endDate=2025-03-31T23:59:59&bbox=40.959,-74.251,40.469,-73.761&pageNum=1&stations=72505394728

### Download data oct-dec 2024 and jan-may 2025
This part will explain the importing data from tlc 
URL_TEMPLATE = https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-09.parquet

In [1]:
# get from Prerequisites Notebook

from urllib.request import urlretrieve
import os

# this is the relative path to the data directory
output_relative_dir = '../data/'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# now, for each type of data set we will need, we will create the paths
for target_dir in ('tlc_data', 'tute_data'): # taxi_zones should already exist
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

In [2]:
URL_TEMPLATE = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_"#year-month.parquet

In [3]:
Y_2024 = '2024' # Year 2024
Y_2025 = '2025' # Year 2025
MONTH_2024 = [9, 10, 11, 12] # Months in 2024
MONTH_2025 = [1, 2, 3, 4] # Months in 2025

In [4]:
# data output directory is `data/tlc_data/`
tlc_output_dir = output_relative_dir + 'tlc_data'

# Year to download data for 2024
for month in MONTH_2024:
    month = str(month).zfill(2) 
    print(f"Begin month {month}")
    url = f'{URL_TEMPLATE}{Y_2024}-{month}.parquet'
    output_dir = f"{tlc_output_dir}/{Y_2024}-{month}.parquet"
    # download
    urlretrieve(url, output_dir) 
    
    print(f"Completed month {month}")
# Year to download data for 2025
for month in MONTH_2025:
    month = str(month).zfill(2) 
    print(f"Begin month {month}") 
    url = f'{URL_TEMPLATE}{Y_2025}-{month}.parquet'
    output_dir = f"{tlc_output_dir}/{Y_2025}-{month}.parquet"
    # download
    urlretrieve(url, output_dir) 
    
    print(f"Completed month {month}")

Begin month 09
Completed month 09
Begin month 10
Completed month 10
Begin month 11
Completed month 11
Begin month 12
Completed month 12
Begin month 01
Completed month 01
Begin month 02
Completed month 02
Begin month 03
Completed month 03
Begin month 04
Completed month 04


In [5]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/25 10:34:01 WARN Utils: Your hostname, TRANG, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/08/25 10:34:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/25 10:34:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/25 10:34:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Preprocess TLC Data

In [6]:
# here, we give it the directory, rather than a specific parquet
# reads all parquet files in the directory
sdf_all = spark.read.parquet('../data/tlc_data/') 

In [ ]:
print("Raw taxi shape:", (sdf_all.count(),
                           len(sdf_all.columns),
                           sdf_all.printSchema()))

Raw taxi shape: (34541965, 19, +--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2025-05-01 00:07:06|  2025-05-01 00:24:15|              1|          3.7|         1|                 N|         140|         202|           1|    

1. Checking for fare_amount that invalid like fare_amount < $3.00  initial charge

In [8]:
sdf_all.filter(sdf_all['fare_amount'] <= 3.00).count() 

1594383

In [9]:
from pyspark.sql import functions as F
# filter invalid fare
# Drop rows where fare_amount < 3.00 or is NULL
taxi_df = sdf_all.filter(
    (F.col("fare_amount") >= 3.00)
)
print("Cleaned taxi shape:", (taxi_df.count(), len(taxi_df.columns)))

Cleaned taxi shape: (33087357, 19)


In [10]:
taxi_df.write.mode("overwrite").parquet("../project-1-individual-phtangtr/data/processed_data/nyc_taxi_cleaned.parquet/")